In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("Bronze-Analysis") \
    .getOrCreate()


In [2]:
df_accidents = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("../raw/Accidents0515.csv")

df_casualties = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("../raw/Casualties0515.csv")

df_vehicles = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("../raw/Vehicles0515.csv")

print("Dados carregados")

Dados carregados


In [3]:
# CÉLULA 3 - Estatísticas Básicas
print("📊 RESUMO DOS DATASETS BRONZE:")
print(f"🚗 Acidentes: {df_accidents.count():,} linhas, {len(df_accidents.columns)} colunas")
print(f"👥 Vítimas: {df_casualties.count():,} linhas, {len(df_casualties.columns)} colunas")  
print(f"🚙 Veículos: {df_vehicles.count():,} linhas, {len(df_vehicles.columns)} colunas")

📊 RESUMO DOS DATASETS BRONZE:
🚗 Acidentes: 1,780,653 linhas, 32 colunas
👥 Vítimas: 2,402,909 linhas, 15 colunas
🚙 Veículos: 3,262,270 linhas, 22 colunas


In [4]:
print("🎯 AMOSTRA - Primeiros registros:")
print("\n--- ACIDENTES ---")
df_accidents.select("Accident_Index", "Accident_Severity", "Date", "Time").show(5)

print("\n--- VÍTIMAS ---")
df_casualties.select("Accident_Index", "Casualty_Severity", "Age_of_Casualty").show(5)

print("\n--- VEÍCULOS ---") 
df_vehicles.select("Accident_Index", "Vehicle_Type", "Age_of_Driver").show(5)

🎯 AMOSTRA - Primeiros registros:

--- ACIDENTES ---
+--------------+-----------------+----------+-------------------+
|Accident_Index|Accident_Severity|      Date|               Time|
+--------------+-----------------+----------+-------------------+
| 200501BS00001|                2|04/01/2005|2025-10-09 17:42:00|
| 200501BS00002|                3|05/01/2005|2025-10-09 17:36:00|
| 200501BS00003|                3|06/01/2005|2025-10-09 00:15:00|
| 200501BS00004|                3|07/01/2005|2025-10-09 10:35:00|
| 200501BS00005|                3|10/01/2005|2025-10-09 21:13:00|
+--------------+-----------------+----------+-------------------+
only showing top 5 rows

--- VÍTIMAS ---
+--------------+-----------------+---------------+
|Accident_Index|Casualty_Severity|Age_of_Casualty|
+--------------+-----------------+---------------+
| 200501BS00001|                2|             37|
| 200501BS00002|                3|             37|
| 200501BS00003|                3|             62|
| 20050